In [1]:
%cd /mnt/hdd1/jkliu/Projects/ATGEmpirical/

/mnt/hdd1/jkliu/Projects/ATGEmpirical


/mnt/hdd1/jkliu/Projects/ATGEmpirical/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
def wrongly_reported(gt_transitions: list, tool_transitions: list):
    return list(set(tool_transitions) - set(gt_transitions))

In [3]:
def missing_transitions(gt_transitions: list, tool_transitions: list):
    return list(set(gt_transitions) - set(tool_transitions))

In [4]:
from result_plot.common import *
from core.apk_util import parse_manifest_file


def single_apk_evaluation(apk_path: str, real_transitions: dict):
    package_name, _, declared_activities, exported_activities = parse_manifest_file(apk_path)
    apk_name, tool_transitions = get_tool_result(apk_path, declared_activities)
    if tool_transitions is None:
        return
    if apk_name not in real_transitions:
        return
    real_transition = real_transitions[apk_name]
    wrongly_reporteds = {}
    missed_transitions = {}
    for tool in tools:
        if tool in tool_transitions:
            tool_transition = tool_transitions[tool]
            wrongly_reporteds[tool] = wrongly_reported(real_transition, tool_transition)
            missed_transitions[tool] = missing_transitions(real_transition, tool_transition)
    return apk_name, real_transition, wrongly_reporteds, missed_transitions


Updated PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/mnt/hdd1/jkliu/Applications/ncurses  # 你自己的 ncurses 目录/bin:/mnt/hdd1/jkliu/Applications/zsh/bin:/mnt/hdd1/jkliu/Applications/Android/cmdline-tools/latest/bin:/mnt/hdd1/jkliu/Applications/Android/emulator:/mnt/hdd1/jkliu/Applications/Android/platform-tools:/mnt/hdd1/jkliu/Applications/Android/tools:/mnt/hdd1/jkliu/Applications/Android/tools/bin:/mnt/hdd1/jkliu/Applications/Android/build-tools/35.0.0
/mnt/hdd1/jkliu/Applications/Android/build-tools/35.0.0/aapt
Successfully added aapt to PATH


In [5]:
apk_path = "/mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.documentfoundation.libreoffice_38.apk"
set_current_version("first_round_no_internet")
apk_name, real_transition, wrongly_reporteds, missed_transitions = single_apk_evaluation(apk_path, get_real_transitions())
print(apk_name)
print(real_transition)
print(wrongly_reporteds)
print(missed_transitions)

extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.documentfoundation.libreoffice_38.apk
/mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.documentfoundation.libreoffice_38.apk
['org.documentfoundation.libreoffice/org.libreoffice.ui.LibreOfficeUIActivity -> org.documentfoundation.libreoffice/org.libreoffice.SettingsActivity', 'org.documentfoundation.libreoffice/org.libreoffice.SettingsActivity -> org.documentfoundation.libreoffice/org.libreoffice.ui.LibreOfficeUIActivity', 'org.documentfoundation.libreoffice/org.libreoffice.ui.LibreOfficeUIActivity -> org.documentfoundation.libreoffice/org.libreoffice.LibreOfficeMainActivity', 'org.documentfoundation.libreoffice/org.libreoffice.LibreOfficeMainActivity -> org.documentfoundation.libreoffice/org.libreoffice.SettingsActivity', 'org.documentfoundation.libreoffice/org.libreoffice.LibreOfficeMainActivity -> org.documentfoundation.libreoffice/org.libreoffice.ui.LibreOfficeUIActivity']
{'qtest': []}
{'qtest': ['o

In [8]:
def evaluate_all_apks(all_apks: list):
    real_transition = get_real_transitions()
    with open("sampled_real_transitions.txt", "w") as f:
        f.write(f"Id,APK,Transition\n")
        counter = 0
        for apk in all_apks:
            if len(real_transition[apk]) > 0:
                for transition in real_transition[apk]:
                    f.write(f"{counter},{apk},{transition}\n")
                counter += 1
    with open("wrongly_reported.txt", "w") as f:
        f.write(f"Id,APK,Tool,Transition,Type\n")
        counter = 1
        for apk in all_apks:
            valid_apk = False
            apk_name, real_transition, wrongly_reporteds, missed_transitions = single_apk_evaluation(apk, real_transitions)
            for tool in tools:
                if tool in wrongly_reporteds:
                    if len(wrongly_reporteds[tool]) > 0:
                        valid_apk = True
                        for transition in wrongly_reporteds[tool]:
                            f.write(f"{counter},{apk},{tool},{transition}\n")
            if valid_apk:
                counter += 1
    with open("missed_transitions.txt", "w") as f:
        f.write(f"Id,APK,Tool,Transition,Type\n")
        counter = 1
        for apk in all_apks:
            valid_apk = False
            apk_name, real_transition, wrongly_reporteds, missed_transitions = single_apk_evaluation(apk, real_transitions)
            for tool in tools:
                if tool in missed_transitions:
                    if len(missed_transitions[tool]) > 0:
                        valid_apk = True
                        for transition in missed_transitions[tool]:
                            f.write(f"{counter},{apk},{tool},{transition}\n")
            if valid_apk:
                counter += 1

In [9]:
import random

real_transitions = get_real_transitions()
# shuffle the list
sampled_apks = list(real_transitions.keys())
random.shuffle(sampled_apks)
# sampled_apks = sampled_apks[:10]
set_current_version("first_round_no_internet")
evaluate_all_apks(sampled_apks)

extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/xyz.myachin.saveto_24.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.darkempire78.opencalculator_42.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.piepmeyer.gauguin_18.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.github.muellerma.mute_reminder_18.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.woheller69.arity_54.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.wmstein.transektcount_401.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.osfans.trime_20240101.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.simplemobiletools.applauncher_53.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.apps.adrcotfas.goodtime_155.apk
extracting summaries